<a href="https://colab.research.google.com/github/m4tuuc/CannModel_en/blob/main/CannModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Este notebook esta todavia en prueba por lo que puede que el performance se vea afectado.

Empezamos instalando lo necesario


In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system

In [ ]:
pip install transformers torch sklearn nlpaug

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install -U transformers tokenizers


**Importamos lo necesario**

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
import torch
import os
import argparse
import sys

device = torch.device('cuda')

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


**Leemos el csv y mapeamos las etiquetas de clase a valores numericos**

Importante tener el archivo csv subido en el Colab

In [3]:
df = pd.read_csv('/content/cannabis.csv')

from pandas.api.types import CategoricalDtype

cat_type = CategoricalDtype(categories=['sativa', 'indica', 'hybrid'], ordered=True)
df['Type'] = df['Type'].astype(cat_type)
print("Valores únicos en 'Type':", df['Type'].unique())
# Mapear a num
label_mapping = {'sativa':0, 'indica':1, 'hybrid':2}
df['labels'] = df['Type'].map(label_mapping).astype(int)


print("Conteo final de clases:")
print(df['labels'].value_counts().sort_index())

Valores únicos en 'Type': ['hybrid', 'sativa', 'indica']
Categories (3, object): ['sativa' < 'indica' < 'hybrid']
Conteo final de clases:
labels
0     440
1     699
2    1212
Name: count, dtype: int64


**Trabajamos con los datos**

Vamos a limpiar los valores nulos despues de esto vamos a convertir toda la descripcion en tipo str.
Haremos uso de la biblioteca de HugginFace para poder trabajar con el csv

In [4]:
df = df.dropna(subset=['Description'])
df['Description'] = df['Description'].astype(str)

#Convertimos los valores de la columna Rating a numerico
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df = df.dropna(subset=['Rating'])


df['Rating'] = df['Rating'].astype(int)

<ipython-input-4-07ff48191aa7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description'] = df['Description'].astype(str)
<ipython-input-4-07ff48191aa7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')


In [5]:
#Guardar el DataFrame limpio en un nuevo CSV
clean_csv_path = '/content/cannabis_clean.csv'
df.to_csv(clean_csv_path, index=False)

#Cargar el datasetcon Hugging Face datasets
dataset = load_dataset('csv', data_files=clean_csv_path)

Generating train split: 0 examples [00:00, ? examples/s]

**Tokenizamos y preparamos todo para el entrenamiento**

Vamos a encargarnos de separa los datos para el entrenamiento y su evaluacion, a continuacion definimos los argumentos con los que vamos a trabajar el modelo

In [6]:
print("Etiquetas únicas:", df['labels'].unique())


num_classes = 3
unique_labels = df['labels'].unique()
if len(unique_labels) < num_classes:
    print(f"Advertencia: Faltan clases. Esperadas: {num_classes}, Encontradas: {len(unique_labels)}")
    all_labels = np.arange(num_classes)
    class_counts = np.array([df[df['labels'] == label].shape[0] if label in unique_labels else 0
                             for label in all_labels])
else:
    class_counts = df['labels'].value_counts().sort_index().values

# Calcular pesos de clase
total_samples = len(df)
class_weights = torch.tensor([total_samples / count for count in class_counts], dtype=torch.float32).to(device)
print("Pesos de clase:", class_weights)

Etiquetas únicas: [2 0 1]
Pesos de clase: tensor([5.3462, 3.3673, 1.9381], device='cuda:0')


In [7]:
# Verifica los valores originales en 'Type'
print("Valores unicos en 'Type':", df['Type'].unique())

# Verifica el mapeo de etiquetas
print("Conteo después del mapeo:")
print(df['Type'].value_counts())
print("Labels mapeados:", df['labels'].value_counts())

Valores únicos en 'Type': ['hybrid', 'sativa', 'indica']
Categories (3, object): ['sativa' < 'indica' < 'hybrid']
Conteo después del mapeo:
Type
hybrid    1211
indica     697
sativa     439
Name: count, dtype: int64
Labels mapeados: labels
2    1211
1     697
0     439
Name: count, dtype: int64


In [8]:
"""
Mejor manejo de funciones de perdidas del modelo
"""

from torch.nn import CrossEntropyLoss
from transformers import Trainer

class CustomTrainer(Trainer):
    def __init__(self, class_weights=None, **kwargs):
        super().__init__(**kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [9]:
#Usamos el tokenizer de BERT
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

def preprocess_function(examples):
    return tokenizer(examples['Description'],padding = "max_length", truncation=True, max_length=256)

#Tokenizar el dataset y nos aseguramos de que 'labels' esten en long y las entradas correctas
tokenized_dataset = dataset.map(preprocess_function, batched=True)

tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


train_test_split = tokenized_dataset['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

Map:   0%|          | 0/2347 [00:00<?, ? examples/s]

In [10]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

train_df, eval_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['labels']
)

# Convertir a Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1877 [00:00<?, ? examples/s]

Map:   0%|          | 0/470 [00:00<?, ? examples/s]

In [11]:
def parse_args():
    parser = argparse.ArgumentParser()

    parser.add_argument('--input_file', type=str, default='/content/cannabis_clean.csv',
                        help='Path to input CSV file')
    parser.add_argument('--output_dir', type=str, default='src/results/final_model',
                        help='Path to save the final model')
    parser.add_argument('--epochs', type=int, default=3,
                        help='Number of training epochs')
    parser.add_argument('--batch_size', type=int, default=8,
                        help='Training batch size')
    parser.add_argument('--eval_batch_size', type=int, default=8,
                        help='Evaluation batch size')
    parser.add_argument('--learning_rate', type=float, default=5e-5,
                        help='Learning rate')

    args, unknown = parser.parse_known_args(sys.argv)

    return args

**Entrenamos el modelo**

In [12]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted'),
        'precision': precision_score(labels, predictions, average='weighted'),
        'recall': recall_score(labels, predictions, average='weighted')
    }

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)
torch.cuda.empty_cache()

In [14]:
def train_model(model, train_dataset, eval_dataset, training_args, class_weights):
    try:
        trainer = CustomTrainer(
            class_weights=class_weights,
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            compute_metrics=compute_metrics,
            tokenizer=tokenizer,
            data_collator=data_collator
        )
        trainer.train()
        return trainer
    except Exception as e:
        print(f"Training failed: {str(e)}")
        raise
def main():
    args = parse_args()

    training_args = TrainingArguments(
    output_dir='./results/checkpoint',
    num_train_epochs=args.epochs,
    per_device_train_batch_size=args.batch_size,
    per_device_eval_batch_size=args.eval_batch_size,
    learning_rate=args.learning_rate,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,  #mixed precision
    dataloader_num_workers=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    gradient_accumulation_steps=2,
    report_to="none"

)


    model = BertForSequenceClassification.from_pretrained(
        'dccuchile/bert-base-spanish-wwm-uncased',
        attn_implementation="eager", # Usar "eager" para ahorrar memoria
        num_labels=3,
        output_attentions=False,
        output_hidden_states=False
    ).to(device)

    # Train
    trainer = train_model(model, tokenized_train, tokenized_eval, training_args, class_weights)

    # Save best model
    best_model_path = args.output_dir
    trainer.save_model(best_model_path)
    print(f"Modelo guardado en {best_model_path}")

if __name__ == '__main__':
    main()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-8-04b198006898>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  super().__init__(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.837300,0.691978,0.742553,0.738809,0.744713,0.742553
2,0.884400,0.574169,0.755319,0.755575,0.761562,0.755319


Model saved to src/results/final_model


In [16]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 58.6 MB/s eta 0:00:00


# Hacer las predicciones

In [23]:
import torch
import numpy as np
import pandas as pd
import faiss
import os
import hashlib
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import Dataset, DataLoader


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LENGTH = 256


def load_model_resources(model_path):
    """Carga el modelo y el tokenizer de forma optimizada."""
    try:
        model = AutoModelForSequenceClassification.from_pretrained(
            model_path,
            torchscript=True  # Esto puede mejorar la inferencia en ciertos casos
        ).to(DEVICE)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        model.eval()
        torch.set_grad_enabled(False)

        print("‡✓‡ Modelo y tokenizer cargados")
        return model, tokenizer

    except Exception as e:
        print(f"❌ Error critico al cargar recursos: {str(e)}")
        return None, None

#Dataset optimizado para la generación de embeddings
class OptimizedStrainDataset(Dataset):
    def __init__(self, descriptions):
        self.descriptions = descriptions

    def __len__(self):
        return len(self.descriptions)

    def __getitem__(self, idx):
        return self.descriptions[idx]

def get_embeddings_hash(df_subset):
    """
    Genera un hash MD5 a partir de los contenidos del dataframe (columnas requeridas)
    para validar la integridad del cache.
    """

    hash_input = df_subset.to_csv(index=False).encode("utf-8") #Convertir a string y codificar para obtener el hash
    return hashlib.md5(hash_input).hexdigest()


def generate_optimized_embeddings(df, model, tokenizer, batch_size=128):
    """
    Genera embeddings a partir de la columna 'Description' del dataframe,
    utilizando paginacion y extrayendo la representación del token [CLS].
    """
    model.eval()
    dataset = OptimizedStrainDataset(df['Description'].tolist())
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    for batch in dataloader:
        inputs = tokenizer(
            batch,
            return_tensors='pt',
            padding='longest',
            truncation=True,
            max_length=MAX_LENGTH
        ).to(DEVICE)

        with torch.inference_mode():
            # Se intenta usar return_dict para extraer last_hidden_state
            try:
                outputs = model.bert(**inputs, return_dict=True)
                hidden_states = outputs.last_hidden_state
            except Exception:
                outputs = model.bert(**inputs)
                hidden_states = outputs[0]  # last_hidden_state en posición 0

        # Extraer el embedding del token [CLS] y mover a CPU
        batch_embeddings = hidden_states[:, 0, :].detach().cpu().numpy()
        embeddings.append(batch_embeddings)

    return np.concatenate(embeddings, axis=0)

#Indice FAISS
def build_optimized_index(embeddings):
    """
    Crea y entrena un indice FAISS para busqueda rapida utilizando inner product.
    Se normalizan los embeddings para utilizar similitud coseno.
    """
    dimension = embeddings.shape[1]
    quantizer = faiss.IndexFlatIP(dimension)
    nlist = min(100, len(embeddings))  # Numero de listas; se ajusta al tamaño del dataset
    index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_INNER_PRODUCT)

    # Normalizar para similitud coseno
    faiss.normalize_L2(embeddings)
    index.train(embeddings)
    index.add(embeddings)

    return index

#Pipeline para la prediccion con FAISS
def optimized_predict(text, model, tokenizer, index, df):
    """
    Realiza la prediccion a partir de un texto de consulta.
    Genera el embedding del texto, lo normaliza y busca la muestra mas cercana en el indice FAISS.
    """
    inputs = tokenizer(
        text,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH
    ).to(DEVICE)

    with torch.inference_mode():
        try:
            outputs = model.bert(**inputs, return_dict=True)
            hidden_states = outputs.last_hidden_state
        except Exception:
            outputs = model.bert(**inputs)
            hidden_states = outputs[0]

    query_embedding = hidden_states[:, 0, :].detach().cpu().numpy()
    faiss.normalize_L2(query_embedding)

    distances, indices = index.search(query_embedding, 1)
    best_idx = indices[0][0]

    raw_description = df.iloc[best_idx]['Description']

    # Limitar la descripción a 250 caracteres, evitando cortar palabras
    # max_chars = 250
    # if len(raw_description) > max_chars:
    #     truncated = raw_description[:max_chars]
    #     last_space = truncated.rfind(' ')
    #     formatted_description = truncated[:last_space] + '...'
    # else:
    #     formatted_description = raw_description

    import textwrap

    line_width = 100  # ancho maximo de cada línea
    formatted_description = textwrap.fill(raw_description, width=line_width)

    return {
        'strain': df.iloc[best_idx]['Type'],
        'confidence': f"{distances[0][0] * 100:.2f}%",
        'description': formatted_description
    }


if __name__ == "__main__":
    # Modificar las rutas si es necesario
    MODEL_PATH = '/content/src/results/final_model'
    DATA_PATH = '/content/cannabis_clean.csv'
    CACHE_DIR = '/content/cache/'

    try:
        # Crear directorio de cache
        os.makedirs(CACHE_DIR, exist_ok=True)

        # Cargar modelo y tokenizer
        model, tokenizer = load_model_resources(MODEL_PATH)
        if not model or not tokenizer:
            raise SystemExit("Error: No se pueden cargar los recursos del modelo.")

        # Cargar dataset y verificar columnas requeridas
        df = pd.read_csv(DATA_PATH)
        required_columns = ['Type', 'Description']
        if not all(col in df.columns for col in required_columns):
            raise ValueError(f"El dataset debe contener las columnas: {required_columns}")

        # Generar hash para la cache
        current_hash = get_embeddings_hash(df[required_columns])
        cache_file = os.path.join(CACHE_DIR, f"embeddings_{current_hash}.npy")

        if os.path.exists(cache_file):
            print("‡✓‡ Cargando embeddings desde caché validado")
            embeddings = np.load(cache_file)
        else:
            print("⚙️ Generando nuevos embeddings")
            embeddings = generate_optimized_embeddings(df, model, tokenizer, batch_size=128)
            np.save(cache_file, embeddings)
            print(f"‡✓‡ Embeddings generados: {embeddings.shape}")

        # FAISS
        index = build_optimized_index(embeddings)

        # Ejemplo de prediccion, colocar la peticion en ingles.
        """
        Para una prediccion mas variada es recomendable hacerlo de forma diferente a la anterior
        ya que esto dara una mayor varieda de respuestas
        """

        test_query = "something to combat my stress"
        result = optimized_predict(test_query, model, tokenizer, index, df)

        # Mostrar resultados
        print("\n" + "=" * 50)
        print(f"🔍 Resultado para: '{test_query}'")
        print(f"⮞Tipo recomendado: {result['strain']}")
        print(f"⮞Nivel de confianza: {result['confidence']}")
        print(f"⮞Descripción:\n{result['description']}")
        print("=" * 50)

    except Exception as e:
        print(f"🔥 Error critico: {str(e)}")
        raise


‡✓‡ Modelo y tokenizer cargados
‡✓‡ Cargando embeddings desde caché validado

🔍 Resultado para: 'something to combat my stress'
⮞Tipo recomendado: indica
⮞Nivel de confianza: 81.62%
⮞Descripción:
Getting its name from the classic bubble gum flavor Gumbo is a perfect medicine for the evenings and
has a smooth taste and finish Gumbo is great for treatment of muscle spasms sleeplessness helpful
with overall pain management as well as serving as an appetite stimulant
